In [21]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.5.0


## データの読み込み

In [2]:
# MNIST データセットを読み込む
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Softmax 回帰によるモデルを作成

softmax(多項ロジスティック)関数は、多変量のロジスティック回帰を利用した多値分類のアルゴリズム。


<img src="./img/softmax.png" style="height: 40px">


Softmax 回帰では、x の重み合計を計算し、バイアスを加え、そして softmax を適用する。
以下のようなイメージ。

* y が目的変数 (0-9の数字)
* x が説明変数 (256 階調のグレースケール、各ピクセルの濃淡を 0.0-1.0 で表したもの)
* W が重み
* b がバイアスの行列

<img src="./img/softmax1.png" style="height: 130px">

式で表すと、

<img src="./img/softmax2.png" style="height: 100px">


今回は、最適な重みとバイアスを推定するための誤差関数（損失関数）として、交差エントロピー (cross entropy) と呼ばれる指標を用います。この交差エントロピーが最も小さくなるように、TensorFlow に `GradientDescentOptimizer` として実装されている、勾配降下法 ( `Gradient descent` ) を用いて学習を進めます。

In [19]:
# 入力データ格納用の 784px 分のプレースホルダを作成
x = tf.placeholder(tf.float32, [None, 784])

# 重み (784 ✕ 10 の行列) の Variable を定義
W = tf.Variable(tf.zeros([784, 10]))

# バイアス (長さ 10 の行列) の Variable を定義
b = tf.Variable(tf.zeros([10]))

# ソフトマックス回帰による予測式を定義
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 出力データ (予測値) 格納用のプレースホルダ
y_ = tf.placeholder(tf.float32, [None, 10])

# 交差エントロピーが最も小さくなるように、学習を行う式を定義
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# 初期化
tf.global_variables_initializer()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
        
# 学習を 1000 回繰り返す
for i in range(1000):
    # 訓練用データから 100 件取得
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # train_spep を実行
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})




## モデルの評価

In [20]:
# 実際の値と予測された値が同じであるか確認
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

# 平均値を求め、予測精度を求める
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# x にテストデータ、 y_ に実際の値をあてはめ、上記で作成した式を実行

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9157
